<a href="https://colab.research.google.com/github/henriquemin/portfolio_data_science/blob/master/Olist_Ecommerce_Sales_Analysis_Plotly_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Olist_Ecommerce_Sales_Analysis Project


## **Business Problem**

Evaluate the Brazilian E-commerce Olist Sales analysis


## **Exploratory Data Analysis (EDA)**
---

### Import Libraries







In [0]:
pip install cufflinks --upgrade
pip install plotly --upgrade

In [0]:
def configure_plotly_browser_state():
  import IPython
  display(IPython.core.display.HTML('''
        <script src="/static/components/requirejs/require.js"></script>
        <script>
          requirejs.config({
            paths: {
              base: '/static/base',
              plotly: 'https://cdn.plot.ly/plotly-latest.min.js?noext',
            },
          });
        </script>
        '''))

In [0]:
from plotly.offline import iplot, plot
import plotly.offline as py
import plotly.graph_objs as go
import cufflinks as cf
cf.go_offline()

In [0]:
import pandas as pd
import numpy as np
import io

### Data preparation & Load file







In [0]:
from google.colab import files

uploaded = files.upload()


Saving olist_classified_public_dataset.csv to olist_classified_public_dataset.csv


In [0]:
df_initial = pd.read_csv(io.BytesIO(uploaded['olist_classified_public_dataset.csv']), header=0, )

In [0]:
df_initial.head()

,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
0,0,1,delivered,89.99,14.38,1,1,2017-08-30 11:41:01.000000,2017-08-30 11:55:08.970352,2017-09-21 00:00:00.000000,2017-09-08 20:35:27.276847,Belo Horizonte,MG,308,beleza_saude,59,492,3,5,NaN,tudo certo,2017-09-09 00:00:00.000000,2017-09-10 03:27:54+00:00,0,0,0,0,0,0,0,0,3,satisfeito,satisfeito_com_pedido
1,1,2,delivered,69.00,15.23,1,1,2017-09-26 09:13:36.000000,2017-09-26 09:28:10.922048,2017-10-24 00:00:00.000000,2017-09-29 21:13:04.984841,Pocos de Caldas,MG,377,brinquedos,50,679,4,5,NaN,o produto chegou antes de prazo parabens pela ...,2017-09-30 00:00:00.000000,2017-10-03 05:34:20+00:00,3,0,0,0,0,0,0,0,0,antes_prazo,satisfeito_com_pedido
2,2,3,delivered,99.80,15.86,2,4,2018-01-15 15:50:42.000000,2018-01-17 07:29:56.419769,2018-02-05 00:00:00.000000,2018-01-23 17:51:31.134866,Sao Jose dos Campos,SP,122,ferramentas_jardim,59,341,2,1,NaN,foi adquirido 6 itens desta loja onde até o mo...,2018-01-24 00:00:00.000000,2018-02-02 17:42:43+00:00,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega
3,3,4,delivered,87.00,12.74,1,1,2018-02-04 11:16:42.000000,2018-02-06 05:31:50.990164,2018-03-13 00:00:00.000000,2018-02-20 19:38:06.633080,Ribeirao Preto,SP,140,informatica_acessorios,45,411,1,4,NaN,achei a entrega um pouco demorada,2018-02-21 00:00:00.000000,2018-02-22 02:09:12+00:00,0,3,0,0,0,0,0,0,0,atrasado,problemas_de_entrega
4,4,5,delivered,99.90,17.95,1,2,2017-12-07 11:58:42.000000,2017-12-08 02:36:49.587515,2018-01-03 00:00:00.000000,2017-12-19 22:33:18.952512,RIO DE JANEIRO,RJ,205,cama_mesa_banho,60,189,1,3,NaN,comprei 02 jogos de lencois de cetim e uma col...,2017-12-20 00:00:00.000000,2017-12-23 04:17:03+00:00,0,0,0,0,0,3,0,0,0,entrega_parcial,problemas_de_entrega



### Identifying Target & Feature variables

In [0]:
# Dataset info
display(df_initial.info())
print('\n')

# Descriptive statistics of the data
display(df_initial.describe())
print('\n')

# statistics for discrete variables
display(df_initial.describe(include=np.object))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3584 entries, 0 to 3583
Data columns (total 34 columns):
Unnamed: 0                       3584 non-null int64
id                               3584 non-null int64
order_status                     3584 non-null object
order_products_value             3584 non-null float64
order_freight_value              3584 non-null float64
order_items_qty                  3584 non-null int64
order_sellers_qty                3584 non-null int64
order_purchase_timestamp         3584 non-null object
order_aproved_at                 3584 non-null object
order_estimated_delivery_date    3584 non-null object
order_delivered_customer_date    3467 non-null object
customer_city                    3584 non-null object
customer_state                   3584 non-null object
customer_zip_code_prefix         3584 non-null int64
product_category_name            3584 non-null object
product_name_lenght              3584 non-null int64
product_description_lenght       

None

,Unnamed: 0,id,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,customer_zip_code_prefix,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied
count,3584.00000,3584.00000,3584.00000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,0.0,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000,3584.000000
mean,1791.50000,1792.50000,133.24183,22.400338,1.101562,1.465960,362.078962,49.092913,744.959542,2.162946,3.522879,NaN,0.404297,0.374442,0.135882,0.133092,0.145089,0.357143,0.119699,0.050781,1.300223
std,1034.75601,1034.75601,210.21544,20.582569,0.451076,1.319876,291.060269,9.907133,641.973898,1.691109,1.657039,NaN,0.919776,0.928583,0.536376,0.504934,0.573524,1.000817,0.473598,0.289724,1.419430
min,0.00000,1.00000,3.50000,0.000000,1.000000,1.000000,10.000000,9.000000,27.000000,1.000000,1.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,895.75000,896.75000,48.00000,13.610000,1.000000,1.000000,124.000000,43.000000,326.000000,1.000000,2.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1791.50000,1792.50000,79.99000,16.790000,1.000000,1.000000,273.000000,52.000000,573.000000,1.000000,4.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,2687.25000,2688.25000,146.99000,23.872500,1.000000,1.000000,595.000000,57.000000,955.000000,3.000000,5.000000,NaN,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
max,3583.00000,3584.00000,4850.00000,401.580000,8.000000,12.000000,998.000000,64.000000,3956.000000,20.000000,5.000000,NaN,3.000000,6.000000,3.000000,4.000000,6.000000,6.000000,3.000000,4.000000,6.000000


,order_status,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,product_category_name,review_comment_message,review_creation_date,review_answer_timestamp,most_voted_subclass,most_voted_class
count,3584,3584,3584,3584,3467,3584,3584,3584,3584,3584,3584,3413,3413
unique,5,3177,3204,257,3090,897,26,60,2812,319,3114,9,3
top,delivered,2017-10-16 14:57:35.000000,2017-10-16 15:07:22.792368,2018-01-12 00:00:00.000000,2017-10-21 12:57:58.486956,Sao Paulo,SP,cama_mesa_banho,muito bom,2017-12-15 00:00:00.000000,2017-09-11 15:46:19+00:00,satisfeito,satisfeito_com_pedido
freq,3467,7,7,31,7,438,1369,471,48,29,10,1491,1983


### Missing Value Treatment
- Treating missing data by imputing with median (continuous, float) & mode (discrete variables, str)

#### Identify null values

In [0]:
# Identify null values on dataset
print ('-' * 10 + " Display information about column types and number of null values " + '-' * 10 )

tab_info = pd.DataFrame(df_initial.dtypes).T.rename(index={0:'column type'})
tab_info = tab_info.append(pd.DataFrame(df_initial.isnull().sum()).T.rename(index={0:'null values'}))
tab_info = tab_info.append(pd.DataFrame(df_initial.isnull().sum()/df_initial.shape[0]*100).T.rename(index={0:'null values (%)'}))

if(any(df_initial.isnull().any())):
    print()
    display(tab_info)
else:
    print('NO missing data')

---------- Display information about column types and number of null values ----------



,Unnamed: 0,id,order_status,order_products_value,order_freight_value,order_items_qty,order_sellers_qty,order_purchase_timestamp,order_aproved_at,order_estimated_delivery_date,order_delivered_customer_date,customer_city,customer_state,customer_zip_code_prefix,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,votes_before_estimate,votes_delayed,votes_low_quality,votes_return,votes_not_as_anounced,votes_partial_delivery,votes_other_delivery,votes_other_order,votes_satisfied,most_voted_subclass,most_voted_class
column type,int64,int64,object,float64,float64,int64,int64,object,object,object,object,object,object,int64,object,int64,int64,int64,int64,float64,object,object,object,int64,int64,int64,int64,int64,int64,int64,int64,int64,object,object
null values,0,0,0,0,0,0,0,0,0,0,117,0,0,0,0,0,0,0,0,3584,0,0,0,0,0,0,0,0,0,0,0,0,171,171
null values (%),0,0,0,0,0,0,0,0,0,0,3.26451,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,4.77121,4.77121


### **Questions to the data**

*   Order status distribution?
*   Monthly sales over year. Which has more sales?
*   Does the freight price increase with the product price?
*   What is the distribution items qty in orders?
*   What is the Customer Classification by satisfaction?




### **Exploring data attributes**

## Columns manipulation


In [0]:
# Transform colunn purchase_timestamp to datetime
df_initial.order_purchase_timestamp = pd.to_datetime(df_initial.order_purchase_timestamp)

In [0]:
# Create a column order_purchase_month to have month format information
df_initial.order_purchase_timestamp.dt.to_period('M')

df_initial['order_purchase_month'] = df_initial.order_purchase_timestamp.dt.to_period('M').astype(str)
df_initial.order_purchase_month.head()

0    2017-08
1    2017-09
2    2018-01
3    2018-02
4    2017-12
Name: order_purchase_month, dtype: object

In [0]:
# Create the month_sales variable
month_sales = df_initial.groupby(by = 'order_purchase_month').order_products_value.sum()

## Data Analysis




### Order status Distribution




In [0]:
# Order status distribution
configure_plotly_browser_state()
df_initial.order_status.value_counts().iplot(kind = 'bar')

# We can see that the majority orders where delivered

### Monthly Sales




In [0]:
# Monthly Sales, line graphic
configure_plotly_browser_state()

data = [go.Scatter(x = month_sales.index,
                   y = month_sales.values)]

py.iplot(data)

# Its an option to see the sales over the time, but we will do it again with bar graphs

In [0]:
# Monthly Sales, bar graphic
configure_plotly_browser_state()

# -----------------------------------------------------------------------------------------------------------------
# Option to cluster by color, sales > mean (lightblue) and sales < mean (red)
# avg_sales = month_sales.values.mean()  # identify the average/mean sales of the period

# colors = []  # Create an empty list for colors

# for i in month_sales.values: # do iteration of each valeu of sales, and if < mean = red, else = lightblue
#  if i < month_sales:
#    colors.append('red')
#  else:
#    colors.append('lightblue')

# -----------------------------------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------------------------------
# Option to highlight the month with major sales withi annotation
max_sales = month_sales.values.max()

colors = []

for x,y in zip(month_sales.values, month_sales.index):
  if x == max_sales:
    month_max_sales = y
    colors.append('blue')
  else:
    colors.append('lightgray')
# -----------------------------------------------------------------------------------------------------------------

# Config of bar graph
trace_01 = [go.Bar(x = month_sales.index,
                   y= month_sales,
                   marker = {'color' : colors,          # to have the color cluster (red/blue), change the 'color' : colors
                             'line' : {'color': '#333',
                                       'width': 1}
                             },
                   opacity = 0.8
                   )
            ]

# Layout creation
config_layout = go.Layout(title = 'MOnthly Sales',
                          yaxis = {'title':'Sales Amount'},
                          xaxis = {'title': 'Period'},
                          font = dict(size=9),
                          annotations = [{'text': 'Highlight month in saes',
                                            'x': month_max_sales,
                                            'y': max_sales}])


# Fig size creation
fig = go.Figure(data = trace_01, layout = config_layout)

# graph plot
py.iplot(trace_01)

# This plot its better to see the sales amount by each month of the year, with highlight in the best sales month.

### Sales by product category




In [0]:
# Sales by product category
category_sales = df_initial.groupby(by='product_category_name').id.count()

# Select category with sales > 100
category_sales = category_sales.loc[category_sales.values >100]

# Descendent order
category_sales.sort_values(ascending= False, inplace = True)

In [0]:
# Sales graphic by category
configure_plotly_browser_state()

trace_02 = go.Bar(y = category_sales.values,
                  x = category_sales.index,
                  marker = {'color': 'lightgreen'},
                  orientation = 'v')

data = [trace_02]

# Layout
config_layout = go.Layout(title = 'Sales by product category',
                          xaxis = dict(
                              titlefont = dict(size=50, color = 'lightgrey'),
                              tickangle = 75),
                          yaxis = { 'title': 'Sales Qty',
                                   'range': [0,max(category_sales.values) * 1.10 ]}, # incrase the y-label to show in the plot
                          font = dict(size=9)
                          )

# fig size
fig = go.Figure(data = trace_02, layout = config_layout)

# graphic plot
py.iplot(data)

# It shows that this dataset has a lot of product sales by differents categories. Its not possible to check if is just 1 ecommerce or more inside the dataset

### Freight Price x Product Price




In [0]:
# Freight price
configure_plotly_browser_state()

# Creating the graph
trace_03 = go.Scatter(x = df_initial['order_freight_value'],
                   y = df_initial['order_products_value'],
                   mode = 'markers',
                   # hover customized text to be shown
                   text = 'Order status: ' + df_initial['order_status'] +
                          '<br>' + 'Class: ' + df_initial['most_voted_class'],
                   # hover exhibition 
                   hoverinfo = 'text+x+y', 
                   marker = {'color': '#941229'})

# Keep the graph in a list
data = [trace_03]

# Layout

config_layout = go.Layout(title = 'Freight price x Product price',
                          yaxis = {'title': 'product price',
                                   'tickformat': '.',
                                   'tickprefix': 'R$ '},
                          xaxis = {'title': 'freight price',
                                   'tickformat': '.',
                                   'tickprefix': 'R$ '})

# Creating a fig
fig = go.Figure(data = trace_03, layout = config_layout)

# Plot the graph
py.iplot(data)

In [0]:
# Freight price x product price by delivery status
configure_plotly_browser_state()

df_initial.iplot(x = 'order_freight_value',
         y = 'order_products_value',
         categories = 'most_voted_subclass',
         title = 'Freight price x Product price by delivery status',
         xTitle = 'Freight price',
         yTitle = 'Product price')

### Distribution Items qty in orders




In [0]:
# Histogram of items by order
configure_plotly_browser_state()

data = [go.Histogram(x = df_initial.order_items_qty)]
layout = go.Layout(title = 'Qty items per Order')
fig = go.Figure(data = data, layout = layout)

py.iplot(fig)

### Customer Classification by satisfaction




In [0]:
# pizza  graph
configure_plotly_browser_state()

# Group by values
most_voted_class = df_initial.groupby(by='most_voted_class').id.count()

# Creating the graph
colors = ['#96D38C', '#FEBFB3','#E1396C']

trace = go.Pie(labels = most_voted_class.index,
              values = most_voted_class.values,
               marker = {'colors': colors,
                         'line': {'color': '#000000', 'width':0.3}},
               hoverinfo = 'label+percent+value',
               pull=[0,0.1,0.1],
               direction = 'clockwise')

data = [trace]

# Layout
config_layout = go.Layout(title = 'Customer Classification by satisfaction')

# Creating the fig
fig = go.Figure(data = data, layout=config_layout)
py.iplot(fig)